In [1]:
!pip install shapely


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import json
import webbrowser
import pandas as pd
import folium
import requests
from shapely.geometry import shape, Point # 지리 공간 데이터 사용

In [2]:
df = pd.read_csv("C:/Sosang/7.트렌드/트렌드.csv", encoding='euc-kr', thousands=',')

In [3]:
df

,구,동,pc검색량,모바일검색량
0,마포구,연남동,12100,108400
1,중구,명동,9940,97960
2,성동구,성수동,9110,80200
3,종로구,익선동,5720,64330
4,용산구,한남동,7140,61300
5,종로구,인사동,4280,46100
6,송파구,방이동,5630,39500
7,서대문구,연희동,3390,30100
8,종로구,삼청동,2290,21600
9,관악구,삼성동,3340,17400


In [4]:
검색_count = df.groupby('구')['모바일검색량'].sum()

In [5]:
mobile=검색_count.sort_values(ascending=False)

In [6]:
mobile = mobile.to_frame()

In [7]:
mobile

,모바일검색량
구,
종로구,151290
중구,133310
마포구,115000
성동구,90200
송파구,73730
용산구,67550
강남구,58250
서대문구,39040
관악구,30450


In [8]:
mobile=pd.DataFrame(mobile)
mobile = mobile.reset_index(drop=False)
mobile

,구,모바일검색량
0,종로구,151290
1,중구,133310
2,마포구,115000
3,성동구,90200
4,송파구,73730
5,용산구,67550
6,강남구,58250
7,서대문구,39040
8,관악구,30450
9,도봉구,20210


In [9]:
mobile

,구,모바일검색량
0,종로구,151290
1,중구,133310
2,마포구,115000
3,성동구,90200
4,송파구,73730
5,용산구,67550
6,강남구,58250
7,서대문구,39040
8,관악구,30450
9,도봉구,20210


In [10]:

# 서울 지도 생성
seoul_map = folium.Map(location=[37.5502, 126.982], zoom_start=12)

# 서울시 행정구역 위도 경도값 GeoJSON 데이터 불러오기
jpath = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'
seoul_geo = json.loads(requests.get(jpath).text) # JSON 문자열을 파이썬 데이터로 변환하여 저장

# GeoJson 객체 생성 및 서울 지도에 추가
folium.GeoJson(
    seoul_geo,
    name='geojson'
).add_to(seoul_map)

# Choropleth 맵 생성 및 추가
folium.Choropleth(
    geo_data=seoul_geo,
    data=mobile,
    columns=['구', '모바일검색량'],
    key_on='feature.properties.name',  # GeoJSON 데이터의 키값 지정 (name이 지역명)
    fill_color='PuBu',
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name='구별 검색량',
    highlight=True
).add_to(seoul_map)


# 데이터프레임의 값을 GeoJSON 데이터에 매핑하여 마커로 표시
for feature in seoul_geo['features']: # 'features'라는 키값 아래의 정보를 추출
    properties = feature['properties']
    city_name = properties['name']
    
    # 데이터프레임에서 해당 지역의 점포수 값 가져오기
    value = mobile[mobile['구'] == city_name]['모바일검색량'].values 
    if len(value) > 0:
        value = value[0]
    else:
        value = 'N/A'
    
    '''
    # GeoJSON 데이터에서 위도와 경도 추출
    coordinates = feature['geometry']['coordinates']
    latitude = coordinates[0][15][1] if coordinates else None
    longitude = coordinates[0][15][0] if coordinates else None
    '''
    
    # 행정구역의 중간 지점 계산
    geom = shape(feature['geometry'])
    center_point = geom.centroid
    
    # 중간 지점에 텍스트로 지역명과 점포수 표시
    folium.Marker(
        location=[center_point.y, center_point.x],
        icon=folium.DivIcon(html=f'<div style="font-size: 12px; color: black; white-space: nowrap;">{city_name}<br/>{value}</div>') 
    ).add_to(seoul_map)
seoul_map
# 지도 저장 및 열기
#seoul_map.save("C:\Sosang\7.트렌드\트렌드.csv",encoding='cp949')
#webbrowser.open_new("C:\Sosang\7.트렌드\트렌드.csv",encoding='cp949')
